In [1]:
%matplotlib inline

import quandl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_datareader.data as web
import pandas as pd
import numpy as np
from talib import RSI, BBANDS, MACD, AROON
import datetime as dt

import plotly.graph_objs as go
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [2]:
max_holding = 100

source = "CSV"

symbol = 'NSE/TCS'

# filter
days = 30 * 12

# frequency
freq = "1D"

### Convert tick data to OHLC

### Import data from CSV/Quandl

In [3]:
if source == "CSV":
    df = pd.read_csv("nse_50.csv")
    df = df.set_index('Date')

    #Convert date from string to date format if required - uncomment below line
    df.index = [dt.datetime.strptime(date, '%Y-%m-%d') for date in df.index]

    rawData = pd.DataFrame({'open':df["Open"],'high':df["High"],'low':df["Low"],'close':df["Close"],'adjClose':df["Adj Close"],"volume":df["Volume"]})


#### Filter Data

In [4]:
#Clean up and filter data
price = rawData[rawData.index> dt.datetime.now() - dt.timedelta(days = days)]
#price = price.iloc[::-1]
price = price.dropna()
close = price["adjClose"].values

#### Resample Data

In [5]:
price = price.resample(freq).agg({'open':{'open':'first'},'high':{'high':'max'},'low':{'low':'min'},'close':{'close':'last'},'adjClose':{'adjClose':'last'},'volume':{'volume':'sum'}})

price.dropna(inplace=True)
price.columns = price.columns.droplevel()


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning:

using a dict with renaming is deprecated and will be removed in a future version



# Machine Learning - Classification model

#### Technical Indicators Calculations

In [6]:
from talib import RSI, BBANDS, MACD, AROON, STOCH, ATR, OBV, ADOSC
#help(ADOSC)

In [7]:
price['macd'], price['macdsignal'], price['macdhist'] = MACD(price.adjClose, fastperiod=12, slowperiod=26, signalperiod=9)

price['RSI'] = RSI(price.adjClose, timeperiod=14)

price['slowk'], price['slowd'] = STOCH(price.high, price.low, price.close)

price['aroondown'], price['aroonup'] = AROON(price.high, price.low)

price['atr'] = ATR(price.high, price.low, price.close)

price['bbup'], price['bbmid'], price['bblow'] = BBANDS(price.close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)

price['obv'] = OBV(np.linspace(1,100,price.index.shape[0]), price.volume)

price['adosc'] = ADOSC(price.high, price.low, price.close, price.volume)

#### Support and resistance calculation

In [8]:
# Pivot points

PP = (price.high+price.low+price.close)/3
R1 = 2 * PP - price.low
S1 = 2 * PP - price.high

R2 = PP + (price.high - price.low)
S2 = PP - (price.high - price.low)

R3 = price.high + 2 * (PP - price.low)
S3 = price.low - 2 * (price.high - PP)

In [9]:
# Fibonacci Pivot

PP = (price.high+price.low+price.close)/3
S1 = PP - 0.382 * (price.high - price.low)
S2 = PP - 0.618 * (price.high - price.low)
S3 = PP -         (price.high - price.low)

R1 = PP + 0.382 * (price.high - price.low)
R2 = PP + 0.618 * (price.high - price.low)
R3 = PP +         (price.high - price.low)

#### Candle stick pattern recognition

In [10]:
from talib import CDLABANDONEDBABY, CDL3BLACKCROWS,CDLDOJI, CDLDOJISTAR, CDLDRAGONFLYDOJI,CDLENGULFING,CDLEVENINGDOJISTAR,CDLEVENINGSTAR, CDLGRAVESTONEDOJI, CDLHAMMER, CDLHANGINGMAN,CDLHARAMI,CDLHARAMICROSS,CDLINVERTEDHAMMER,CDLMARUBOZU,CDLMORNINGDOJISTAR,CDLMORNINGSTAR,CDLSHOOTINGSTAR,CDLSPINNINGTOP,CDL3BLACKCROWS, CDL3LINESTRIKE

price[CDLABANDONEDBABY(price.open, price.high, price.low, price.close) != 0]

,open,high,low,close,adjClose,volume,macd,macdsignal,macdhist,RSI,slowk,slowd,aroondown,aroonup,atr,bbup,bbmid,bblow,obv,adosc


In [11]:
#https://github.com/mrjbq7/ta-lib

CDLABANDONEDBABY(price.open, price.high, price.low, price.close)

CDL3BLACKCROWS(price.open, price.high, price.low, price.close)

CDLDOJI(price.open, price.high, price.low, price.close)

CDLDOJISTAR(price.open, price.high, price.low, price.close)

CDLDRAGONFLYDOJI(price.open, price.high, price.low, price.close)

CDLENGULFING(price.open, price.high, price.low, price.close)

CDLEVENINGDOJISTAR(price.open, price.high, price.low, price.close)

CDLEVENINGSTAR(price.open, price.high, price.low, price.close)

CDLGRAVESTONEDOJI(price.open, price.high, price.low, price.close)

CDLHAMMER(price.open, price.high, price.low, price.close)

CDLHANGINGMAN(price.open, price.high, price.low, price.close)

CDLHARAMI(price.open, price.high, price.low, price.close)

CDLHARAMICROSS(price.open, price.high, price.low, price.close)

CDLINVERTEDHAMMER(price.open, price.high, price.low, price.close)

CDLMARUBOZU(price.open, price.high, price.low, price.close)

CDLMORNINGDOJISTAR(price.open, price.high, price.low, price.close)

CDLMORNINGSTAR(price.open, price.high, price.low, price.close)

CDLSHOOTINGSTAR(price.open, price.high, price.low, price.close)


CDLSPINNINGTOP(price.open, price.high, price.low, price.close)

CDL3BLACKCROWS(price.open, price.high, price.low, price.close)

CDL3LINESTRIKE(price.open, price.high, price.low, price.close)

2018-04-03    0
2018-04-04    0
2018-04-05    0
2018-04-06    0
2018-04-09    0
2018-04-10    0
2018-04-11    0
2018-04-12    0
2018-04-13    0
2018-04-16    0
2018-04-17    0
2018-04-18    0
2018-04-19    0
2018-04-20    0
2018-04-23    0
2018-04-24    0
2018-04-25    0
2018-04-26    0
2018-04-27    0
2018-04-30    0
2018-05-02    0
2018-05-03    0
2018-05-04    0
2018-05-07    0
2018-05-08    0
2018-05-09    0
2018-05-10    0
2018-05-11    0
2018-05-14    0
2018-05-15    0
             ..
2019-02-05    0
2019-02-06    0
2019-02-07    0
2019-02-08    0
2019-02-11    0
2019-02-12    0
2019-02-14    0
2019-02-15    0
2019-02-18    0
2019-02-19    0
2019-02-20    0
2019-02-21    0
2019-02-22    0
2019-02-25    0
2019-02-26    0
2019-02-27    0
2019-02-28    0
2019-03-01    0
2019-03-05    0
2019-03-06    0
2019-03-07    0
2019-03-08    0
2019-03-11    0
2019-03-12    0
2019-03-13    0
2019-03-14    0
2019-03-15    0
2019-03-18    0
2019-03-19    0
2019-03-20    0
Length: 239, dtype: int3

#### TODO: BUY/SELL decisions

In [12]:
from enum import Enum
class S(Enum):
    BUY = 1
    SELL = 2
    HOLD = 3
    
price.dropna(inplace=True)

#price.drop(columns=['prevClose'], inplace=True)

#print(price.head(5))

#price['Delta'] = price['prevClose']/price['close'] - 1

labels = pd.DataFrame(data = {"holdings":np.zeros(price.index.shape[0])},index=price.index)

#labels[price['Delta']>0] = S.BUY
#labels[price['Delta']<=0] = S.SELL

#price['Holdings'] = labels

labels[(price['macdhist'] > price['macdhist'].shift(1)) & (price['macdhist'].shift(1) < 0 ) & (price['macdhist'] >= 0 )] = S.BUY
labels[(price['macdhist'] < price['macdhist'].shift(1)) & (price['macdhist'].shift(1) > 0 ) & (price['macdhist'] <= 0 )] = S.SELL

In [13]:
# buy and sell decisions are stored in arrays for plotting purpose(visualisation)

buy = price[labels['holdings']==S.BUY]
sell = price[labels['holdings']==S.SELL]

# Plotting Results

In [14]:
# Price vs date curve with green and red dots to indicate buy and sell points

trace0 = go.Scatter(x=buy.index, y=buy.adjClose, marker = dict(color='green', size=8), mode='markers', name="Buy")
trace1 = go.Scatter(x=sell.index, y=sell.adjClose, marker = dict(color='red', size=8), mode='markers', name="Sell")

# Closing Price line chart
trace2 = go.Scatter(x=price.index, y=price.adjClose, marker = dict(color='grey', size=2), mode='lines', name="Close Price", yaxis='y1')

# Candlestick
traceOHLC = go.Candlestick(x=price.index, open=price['open'], high=price['high'], low=price['low'], close=price['close'], name="Candlestick")


# BBP
traceBBU = go.Scatter(x=price.index, y=price['bbup'], name='BB_up',  line=dict(color='lightgrey'))
traceBBL = go.Scatter(x=price.index, y=price['bblow'], name='BB_low',  line=dict(color='lightgrey'), fill = 'tonexty', fillcolor="rgba(0,40,100,0.02)")


# RSI plots
traceRSI = go.Scatter(x=price.index, y=price['RSI'],mode='lines', line=dict(color='rgb(63, 72, 204)'), name='RSI')

# ATR
traceATR = go.Scatter(x=price.index, y=price['atr'],mode='lines', line=dict(color='rgb(63, 72, 204)'), name='ATR')


# MACD plots
traceMACD = go.Scatter(x=price.index, y=price['macd'], name='MACD', line=dict(color='blue'))
traceMACDHist = go.Bar(x=price.index, y=price['macdhist'], name='MACD Hist', marker=dict(color="lightgrey"))
traceMACDSignal = go.Scatter(x=price.index, y=price['macdsignal'], name='MACD signal', line=dict(color='red'))

# Volume Chart
traceV = go.Bar(x=price.index, y=price.volume, name='Volume', marker=dict(color='lightgrey'),yaxis='y2')
traceOBV = go.Scatter(x=price.index, y=price['obv'], name='OBV', line=dict(color='black'))



# Stochastics

traceSK  = go.Scatter(x=price.index, y=price['slowk'], name='%K', line=dict(color='black'), yaxis='y3')
traceSD  = go.Scatter(x=price.index, y=price['slowd'], name='%D', line=dict(color='red'))

## AROON
traceARD  = go.Scatter(x=price.index, y=price['aroondown'], name='AROON Down', line=dict(color='red'), yaxis='y4')
traceARU  = go.Scatter(x=price.index, y=price['aroonup'], name='AROON UP', line=dict(color='green'))


fig = tools.make_subplots(rows=7, cols=1, row_width=[1,1,2,3,1,4,4])

# OHLC
fig.append_trace(traceOHLC, 1, 1)

# Price
fig.append_trace(trace0, 2, 1)
fig.append_trace(trace1, 2, 1)
fig.append_trace(trace2, 2, 1)

#BB
fig.append_trace(traceBBU, 2, 1)
fig.append_trace(traceBBL, 2, 1)

#Volume
fig.append_trace(traceV,3,1)
#fig.append_trace(traceOBV,2,1)

# MACD
fig.append_trace(traceMACD, 4, 1)
fig.append_trace(traceMACDHist, 4, 1)
fig.append_trace(traceMACDSignal, 4, 1)

# RSI
#fig.append_trace(traceRSI, 6, 1)

# ATR
fig.append_trace(traceATR, 6, 1)


# Stochastics
fig.append_trace(traceSK, 5, 1)
fig.append_trace(traceSD, 5, 1)

# AARON
fig.append_trace(traceARD, 7, 1)
fig.append_trace(traceARU, 7, 1)



# Update layout
fig['layout'].update(height=1150, plot_bgcolor='rgba(0,0,0,0)')

fig['layout']['xaxis']['rangeslider'].update(visible=False)
fig['layout'].update(yaxis = dict(side='right', domain=[0.75,1]), 
                     yaxis2 = dict(side='right', domain=[0.48,0.75]), 
                    xaxis  = dict(visible=False),
                    xaxis2  = dict(visible=False),
                    yaxis3 = dict(side='left', showgrid=False, title='Volume'),
                    yaxis4 = dict(side='right', title='MACD'),
                    yaxis5 = dict(side='right', title='Stochastics'),
                    yaxis6 = dict(side='right', title='ATR'),
                    yaxis7 = dict(side='right', title='AROON'))

rectLayout =  [
        # 1st highlight during Feb 4 - Feb 6
        {
            'type': 'rect',
            # x-reference is assigned to the x-values
            'xref': 'x',
            # y-reference is assigned to the plot paper [0,1]
            'yref': 'y4',
            'x0': price.index[0],
            'y0': 20,
            'x1': price.index[-1],
            'y1': 80,
            'fillcolor': '#d3d3d3',
            'opacity': 0.4,
            'line': {
                'width': 0,
            }
        }]
fig['layout'].update(shapes = rectLayout)


                     
iplot(fig, filename="Test")

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]
[ (4,1) x4,y4 ]
[ (5,1) x5,y5 ]
[ (6,1) x6,y6 ]
[ (7,1) x7,y7 ]

